# EOD: load index JSONs

- EOD docs indicate lots of ticker data: https://eodhistoricaldata.com/api/eod/MCD.US?api_token=6057885c018fd9.09873427
- this notebook is for loading index JSONS (e.g. gspd). These index JSON files can be used by the fundamentals load notebook https://github.com/ryanmark1867/stock_investigation/blob/master/notebooks/eod_fundamentals_load.ipynb to get the fundamentals for individual tickers listed in index JSON files


In [15]:
import pandas as pd
import numpy as np
import os
import requests
import json
from io import StringIO
import yaml
from datetime import date
import requests
from sklearn.metrics import classification_report


# For reading stock data from yahoo
#import pandas_datareader as pdr
from pandas_datareader.data import DataReader
# import yahoo_fin.stock_info as si

# For time stamps
from datetime import datetime

# for LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
from keras.models import Model
# from tensorflow.keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
# access datasets from quandl.com - need to pip install Quandl to use
import quandl
config_file = 'eod_test_config.yml'

In [16]:
# load config file
current_path = os.getcwd()
print("current directory is: "+current_path)

path_to_yaml = os.path.join(current_path, config_file)
print("path_to_yaml "+path_to_yaml)
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file')

current directory is: C:\personal\karma_stocks_2021\stock_investigation\notebooks
path_to_yaml C:\personal\karma_stocks_2021\stock_investigation\notebooks\eod_test_config.yml


In [17]:
# load config parms
parms = {}
eod_token = config['general']['eod_token']
eod_historical_token = config['general']['eod_historical_token']
fundamentals_directory = config['files']['fundamentals_directory']

from_date = config['general']['master_start']
to_date = config['general']['master_end']
parms['master_date_mode'] = config['general']['master_date_mode']
parms['master_start'] = config['general']['master_start']
parms['master_end'] = config['general']['master_end']

In [18]:
# DataReader: https://riptutorial.com/pandas/topic/1912/pandas-datareader
# https://pandas-datareader.readthedocs.io/en/latest/remote_data.html
# 
# Set up End and Start times for data grab
# check to see if start and end dates are hard-coded with master dates
def set_start_end():
    if parms['master_date_mode']: # start and end hardcoded by parameters
        start = parms['master_start']
        end = parms['master_end']
    else: # end is current date; start is current date minus years_window
        end = datetime.now()
        start = datetime(end.year - parms['years_window'], end.month, end.day)
    # output a test dataset
    tester = DataReader('IBM', 'stooq', start, end)
    print(tester.shape)
    return(start,end)

In [19]:
def get_path():
    rawpath = os.getcwd()
    # data is in a directory that is a sibling to the directory containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', fundamentals_directory))
    return(path)

# Experiment getting from EOD historical data
https://eodhistoricaldata.com/cp/settings
https://eodhistoricaldata.com/api/eod/MCD.US?from=2020-01-05&to=2020-02-10&period=d&api_token=6057885c018fd9.09873427

- documentation for API https://eodhistoricaldata.com/financial-apis/api-for-historical-data-and-volumes/
- https://eodhistoricaldata.com/api/eod/MCD.US?api_token=6057885c018fd9.09873427



- MCD.US consists of two parts: {SYMBOL_NAME}.{EXCHANGE_ID}, then you can use, for example, MCD.MX for Mexican Stock Exchange. or MCD.US for NYSE. Check the list of supported exchanges to get more information about stock markets we do support.

- api_token – your own API KEY, which you will get after you subscribe to our services.
- fmt – the output format. Possible values are ‘csv’ for CSV output and ‘json’ for JSON output. Default value: ‘csv’.
- period – use ‘d’ for daily, ‘w’ for weekly, ‘m’ for monthly prices. By default, daily prices will be shown.
- order – use ‘a’ for ascending dates (from old to new), ‘d’ for descending dates (from new to old). By default, dates are shown in ascending order.
- from and to – the format is ‘YYYY-MM-DD’. If you need data from Jan 5, 2017, to Feb 10, 2017, you should use from=2017-01-05 and to=2017-02-10.

In [20]:
# function to get data from EOD historical API
def get_eod_close_data(symbol='AAPL.US', api_token='OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX', from_date = '2021-07-01', to_date = '2021-07-08'):
    session = requests.Session()
    base_url = 'https://eodhistoricaldata.com/api/eod/'+symbol
    url = 'https://eodhistoricaldata.com/api/div/%s' % symbol
    params = {'api_token': api_token,'from':from_date,'to':to_date}
    r = session.get(base_url, params=params)
    if r.status_code == requests.codes.ok:
        df = pd.read_csv(StringIO(r.text), skipfooter=0, parse_dates=[0], index_col=0, engine='python')
        return(True, df)
    else:
        print("status code",str(r.status_code))
        print("reason code",str(r.reason))
        return(False,"null")
        #raise Exception(r.status_code, r.reason, url)

In [21]:


# https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX
def get_eod_ticker_details(symbol='AAPL.US', api_token='OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX'):
    session = requests.Session()
    base_url = 'https://eodhistoricaldata.com/api/fundamentals/'+symbol
    params = {'api_token': api_token}
    r = session.get(base_url, params=params)
    if r.status_code == requests.codes.ok:
        df = pd.read_csv(StringIO(r.text), skipfooter=0,index_col=0)
        return(True, r)
    else:
        print("status code",str(r.status_code))
        print("reason code",str(r.reason))
        return(False,"null")
        #raise Exception(r.status_code, r.reason, url)

In [22]:
# get the JSON fundamentals dump for a given ticker

# https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX
def get_eod_ticker_fundamentals(symbol='AAPL.US', api_token='OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX'):
    session = requests.Session()
    base_url = 'https://eodhistoricaldata.com/api/fundamentals/'+symbol
    params = {'api_token': api_token}
    r = session.get(base_url, params=params)
    if r.status_code == requests.codes.ok:
        df = pd.read_csv(StringIO(r.text), skipfooter=0,index_col=0)
        return(True, r)
    else:
        print("status code",str(r.status_code))
        print("reason code",str(r.reason))
        return(False,"null")
        #raise Exception(r.status_code, r.reason, url)

In [23]:
def write_json_file(ticker,output_path,json_struct):
    str_date = datetime.now().strftime("%Y-%m-%d")
    file_name = os.path.join(output_path,ticker+'_'+str_date+'.json')
    print("file name is: ",file_name)
    #json_object = json.dumps(json_struct, indent = 4)
    json_object = json_struct
    with open(file_name, 'w') as outfile:
        json.dump(json_object, outfile)
    #with open(file_name, "w") as outfile:
    #    outfile.write(json_object)

In [24]:
# main block

#company = 'IBM.US'
#company = 'GSPC.INDX'
company = "W5KLCV.INDX"
token = eod_token
print("company is: ",company)
result_stat, r = get_eod_ticker_fundamentals(company,eod_token)
# clean off the backslashes and starting and ending double quotes
r_text = r.text
json_output = json.loads(r.text)
no_backslash = r_text.replace("\\", "")
# my_str[:-1]
no_end_quote = no_backslash[:-1]
prepped_json = no_end_quote[1:]
write_json_file('w5klcv',get_path(),json_output)


company is:  W5KLCV.INDX
file name is:  C:\personal\karma_stocks_2021\stock_investigation\static_load_fundamentals\w5klcv_2021-10-14.json


In [25]:
'''
# with open("sample.json", "w") as outfile:
#    outfile.write(json_object)

json_object = json.dumps(r.text, indent = 4)
with open("spy.json", "w") as outfile:
    outfile.write(json_object)
    '''

'\n# with open("sample.json", "w") as outfile:\n#    outfile.write(json_object)\n\njson_object = json.dumps(r.text, indent = 4)\nwith open("spy.json", "w") as outfile:\n    outfile.write(json_object)\n    '

In [26]:
'''
# brute force call 
# https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX
session = requests.Session()
base_url = "https://eodhistoricaldata.com/api/fundamentals/GSPC.INDX"
api_token = "605766cf49ac93.89487471"
params = {'api_token': api_token}
r = session.get(base_url, params=params)
if r.status_code == requests.codes.ok:
    #df = pd.read_csv(StringIO(r.text), skipfooter=0, parse_dates=[0], index_col=0, engine='python')
    df = pd.read_csv(StringIO(r.text), skipfooter=0,index_col=0)

    
else:
    print("status code",str(r.status_code))
    print("reason code",str(r.reason))
    '''


'\n# brute force call \n# https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX\nsession = requests.Session()\nbase_url = "https://eodhistoricaldata.com/api/fundamentals/GSPC.INDX"\napi_token = "605766cf49ac93.89487471"\nparams = {\'api_token\': api_token}\nr = session.get(base_url, params=params)\nif r.status_code == requests.codes.ok:\n    #df = pd.read_csv(StringIO(r.text), skipfooter=0, parse_dates=[0], index_col=0, engine=\'python\')\n    df = pd.read_csv(StringIO(r.text), skipfooter=0,index_col=0)\n\n    \nelse:\n    print("status code",str(r.status_code))\n    print("reason code",str(r.reason))\n    '

In [27]:
'''
# with open("sample.json", "w") as outfile:
#    outfile.write(json_object)

json_object = json.dumps(r.text, indent = 4)
with open("gspc.json", "w") as outfile:
    outfile.write(json_object)
    '''

'\n# with open("sample.json", "w") as outfile:\n#    outfile.write(json_object)\n\njson_object = json.dumps(r.text, indent = 4)\nwith open("gspc.json", "w") as outfile:\n    outfile.write(json_object)\n    '